In [ ]:
# =========================================================
# DERM7PT DATA LOADER FOR TRAINING NOTEBOOKS
# =========================================================
import os
import json
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# =========================================================
# PATHS TO PREPROCESSED DATA
# =========================================================
PREPROCESSED_DIR = r"augmented"

TRAIN_CSV = os.path.join(PREPROCESSED_DIR, "train_metadata_final.csv")
VAL_CSV   = os.path.join(PREPROCESSED_DIR, "val_metadata_final.csv")
TEST_CSV  = os.path.join(PREPROCESSED_DIR, "test_metadata_final.csv")

INFO_PATH = os.path.join(PREPROCESSED_DIR, "preprocessing_info.json")

# =========================================================
# LOAD PREPROCESSED DATA
# =========================================================
print("Loading preprocessed Derm7pt data...")

# Load CSVs
train_df = pd.read_csv(TRAIN_CSV)
val_df   = pd.read_csv(VAL_CSV)
test_df  = pd.read_csv(TEST_CSV)

# Load preprocessing info
with open(INFO_PATH, "r") as f:
    preprocessing_info = json.load(f)

categorical_cols = preprocessing_info["categorical_cols"]
label_mapping = preprocessing_info["label_mapping"]

# Extract class names from label_mapping (sorted by label index)
class_names = [k for k, v in sorted(label_mapping.items(), key=lambda x: x[1])]

print(f"\n✅ Training samples:   {len(train_df)}")
print(f"✅ Validation samples: {len(val_df)}")
print(f"✅ Test samples:       {len(test_df)}")
print(f"\nLabel mapping: {label_mapping}")
print(f"Class names: {class_names}")

# =========================================================
# EXTRACT FEATURES AND LABELS
# =========================================================
def extract_features(df):
    """Extract image paths, metadata features, and labels from dataframe"""
    img_paths = df["ImagePath"].values
    labels = df["label"].values
    
    # Metadata features (all columns except ImagePath and label)
    metadata_cols = [col for col in df.columns if col not in ["ImagePath", "label"]]
    metadata = df[metadata_cols].values
    
    return img_paths, metadata, labels

X_train_img, X_train_meta, y_train = extract_features(train_df)
X_val_img, X_val_meta, y_val       = extract_features(val_df)
X_test_img, X_test_meta, y_test    = extract_features(test_df)

num_classes = len(label_mapping)
print(f"\nNumber of classes: {num_classes}")

# Get input dimensions
non_feature_cols = ["label", "ImagePath"]
X_train_meta_df = train_df.drop(columns=non_feature_cols)
input_dim_meta = X_train_meta_df.shape[1]
print(f"Metadata input dimension: {input_dim_meta}")

# =========================================================
# PYTORCH DATASET CLASS
# =========================================================
class Derm7ptDataset(Dataset):
    """
    Custom Dataset for Derm7pt with images + metadata
    """
    def __init__(self, img_paths, metadata, labels, transform=None):
        self.img_paths = img_paths
        self.metadata = metadata
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.img_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # Fallback to black image if loading fails
            print(f"Warning: Failed to load {img_path}, using placeholder")
            image = Image.new("RGB", (224, 224), color="black")
        
        if self.transform:
            image = self.transform(image)
        
        # Get metadata and label
        metadata = self.metadata[idx].astype(np.float32)
        label = int(self.labels[idx])
        
        return image, metadata, label

# =========================================================
# DATA TRANSFORMS
# =========================================================
# Training transforms (with augmentation)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation/Test transforms (no augmentation)
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# =========================================================
# CREATE DATASETS
# =========================================================
train_dataset = Derm7ptDataset(X_train_img, X_train_meta, y_train, transform=train_transform)
val_dataset   = Derm7ptDataset(X_val_img, X_val_meta, y_val, transform=val_test_transform)
test_dataset  = Derm7ptDataset(X_test_img, X_test_meta, y_test, transform=val_test_transform)

print(f"\n✅ Created PyTorch Datasets")
print(f"   - Train: {len(train_dataset)} samples")
print(f"   - Val:   {len(val_dataset)} samples")
print(f"   - Test:  {len(test_dataset)} samples")

# =========================================================
# CREATE DATALOADERS
# =========================================================
BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,  # Set to 0 for Windows
    pin_memory=True)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print(f"\n✅ Created DataLoaders (batch_size={BATCH_SIZE})")
print(f"   - Train batches: {len(train_loader)}")
print(f"   - Val batches:   {len(val_loader)}")
print(f"   - Test batches:  {len(test_loader)}")

# =========================================================
# EXAMPLE: TEST LOADING A BATCH
# =========================================================
print("\n🔍 Testing batch loading...")
for images, metadata, labels in train_loader:
    print(f"   - Image batch shape:    {images.shape}")
    print(f"   - Metadata batch shape: {metadata.shape}")
    print(f"   - Labels batch shape:   {labels.shape}")
    break

print("\n✅ Derm7pt data loading complete! Ready for training.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_cluster import knn_graph
import timm

class EarlyFusionWithMLPMeta(nn.Module):
    def __init__(self, input_dim_meta, num_classes):
        super().__init__()

        # === Pure MLP encoder for metadata ===
        self.meta_embed = nn.Sequential(
            nn.Linear(input_dim_meta, 64 * 64),
            nn.ReLU(),
            nn.BatchNorm1d(64 * 64),
            nn.Dropout(0.3)
        )

        # Reduce to 32D for meta→image
        self.meta_reduce = nn.Linear(64 * 64, 32)

        # === Metadata → pseudo-image ===
        self.meta_to_image = nn.Sequential(
            nn.Linear(32, 56 * 56),
            nn.ReLU(),
            nn.BatchNorm1d(56 * 56),
            nn.Dropout(0.3)
        )

        # === MobileViT backbone (same as original) ===
        self.mobilevit = timm.create_model("mobilevit_s.cvnets_in1k", pretrained=True, num_classes=0)

        stem_conv = self.mobilevit.stem.conv
        new_conv = nn.Conv2d(4, stem_conv.out_channels,
                             kernel_size=stem_conv.kernel_size,
                             stride=stem_conv.stride,
                             padding=stem_conv.padding)
        with torch.no_grad():
            new_conv.weight[:, :3] = stem_conv.weight
            new_conv.weight[:, 3:] = stem_conv.weight.mean(dim=1, keepdim=True) * 0.1
        self.mobilevit.stem.conv = new_conv

        self.mobilevit.stages = nn.Sequential(*list(self.mobilevit.stages.children())[:4])
        self.mobilevit.final_conv = nn.Identity()
        self.mobilevit.head = nn.Identity()

        self.post_conv = nn.Sequential(
            nn.Conv2d(128, 160, kernel_size=1, bias=False),
            nn.BatchNorm2d(160),
            nn.ReLU(inplace=True)
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(160, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, img, meta, batch_idx=None):
        B = meta.size(0)

        # === MLP metadata encoder ===
        meta_emb = self.meta_embed(meta)          # [B,4096]
        meta_emb = self.meta_reduce(meta_emb)     # [B,32]

        # === metadata → pseudo-image ===
        meta_img = self.meta_to_image(meta_emb).view(B, 1, 56, 56)
        meta_img = F.interpolate(meta_img, size=(224, 224), mode='bilinear')

        # === early fusion ===
        x = torch.cat([img, meta_img], dim=1)

        # === MobileViT ===
        x = self.mobilevit.stem(x)
        x = self.mobilevit.stages(x)
        x = self.post_conv(x)
        x = self.pool(x).view(B, -1)

        return self.classifier(x)


from torchinfo import summary

batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EarlyFusionWithMLPMeta(input_dim_meta, num_classes).to(device)

dummy_img = torch.randn(batch_size, 3, 224, 224).to(device)
dummy_meta = torch.randn(batch_size, input_dim_meta).to(device)
dummy_batch_idx = torch.arange(batch_size).to(device)

summary(
    model,
    input_data=[dummy_img, dummy_meta, dummy_batch_idx],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    depth=3
)

In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# =========================================================
# Utility Functions
# =========================================================
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

# =========================================================
# Training Function
# =========================================================
def train_model(model, train_loader, val_loader, device, epochs=100, patience=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        
        for images, metadata, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, metadata, labels = images.to(device), metadata.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images, metadata)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        train_accuracy = correct / total
        history["train_loss"].append(train_loss / len(train_loader))
        history["train_acc"].append(train_accuracy)
        
        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, metadata, labels in val_loader:
                images, metadata, labels = images.to(device), metadata.to(device), labels.to(device)
                outputs = model(images, metadata)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(labels).sum().item()
                total += labels.size(0)
        
        val_accuracy = correct / total
        avg_val_loss = val_loss / len(val_loader)
        history["val_loss"].append(avg_val_loss)
        history["val_acc"].append(val_accuracy)
        
        print(f"Epoch {epoch+1}: Train Loss={history['train_loss'][-1]:.4f}, Train Acc={train_accuracy:.4f}")
        print(f"           Val Loss={avg_val_loss:.4f}, Val Acc={val_accuracy:.4f}")
        
        # Early Stopping
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
        
        scheduler.step(avg_val_loss)
    
    return best_model_state, history, best_val_accuracy

# =========================================================
# Evaluation Function
# =========================================================
def evaluate_model(model, test_loader, device):
    model.eval()
    all_labels, all_preds = [], []
    
    with torch.no_grad():
        for images, metadata, labels in test_loader:
            images, metadata = images.to(device), metadata.to(device)
            outputs = model(images, metadata)
            preds = outputs.argmax(dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test F1 Score: {f1:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    
    return accuracy, f1, precision, recall

# =========================================================
# Multi-Seed Training
# =========================================================
seeds = [42, 123, 569]
best_overall_model = None
best_overall_accuracy = 0.0

results = {"accuracy": [], "f1": [], "precision": [], "recall": []}
all_histories = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

for seed in seeds:
    print(f"\n{'='*60}")
    print(f"Training with Seed {seed}")
    print(f"{'='*60}")
    set_seed(seed)
    
    model = EarlyFusionWithMLPMeta(input_dim_meta=input_dim_meta, num_classes=num_classes).to(device)
    
    best_model_state, history, val_acc = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        epochs=100,
        patience=5
    )
    
    # Store history
    all_histories["train_loss"].append(history["train_loss"])
    all_histories["val_loss"].append(history["val_loss"])
    all_histories["train_acc"].append(history["train_acc"])
    all_histories["val_acc"].append(history["val_acc"])
    
    # Evaluate on test set
    model.load_state_dict(best_model_state)
    acc, f1, prec, recall = evaluate_model(model, test_loader, device)
    
    results["accuracy"].append(acc)
    results["f1"].append(f1)
    results["precision"].append(prec)
    results["recall"].append(recall)
    
    if val_acc > best_overall_accuracy:
        best_overall_accuracy = val_acc
        best_overall_model = best_model_state

# Save best model
torch.save(best_overall_model, "Derm7ptMLPFusion_best.pth")
print(f"\n✅ Best model saved with Val Accuracy: {best_overall_accuracy:.4f}")

# Summary
print("\n" + "="*60)
print("Final Results Across All Seeds")
print("="*60)
for metric in results:
    mean_val = np.mean(results[metric])
    std_val = np.std(results[metric])
    print(f"{metric.capitalize()}: {mean_val:.4f} ± {std_val:.4f}")

In [ ]:
print("\n" + "="*60)
print("Final Results Across All Seeds")
print("="*60)
for metric in results:
    mean_val = np.mean(results[metric])
    std_val = np.std(results[metric])
    print(f"{metric.capitalize()}: {mean_val:.4f} ± {std_val:.4f}")

In [ ]:
import matplotlib.pyplot as plt

def plot_average_training_curves(all_histories, save_path="MLPFusion_training_curves.png"):
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['axes.titleweight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    
    # Find minimum length
    min_len = min(len(h) for h in all_histories["train_loss"])
    
    # Truncate to same length
    train_loss = np.array([h[:min_len] for h in all_histories["train_loss"]])
    val_loss = np.array([h[:min_len] for h in all_histories["val_loss"]])
    train_acc = np.array([h[:min_len] for h in all_histories["train_acc"]])
    val_acc = np.array([h[:min_len] for h in all_histories["val_acc"]])
    
    epochs = np.arange(1, min_len + 1)
    
    plt.figure(figsize=(12, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss.mean(axis=0), label="Train Loss")
    plt.fill_between(epochs,
                     train_loss.mean(axis=0) - train_loss.std(axis=0),
                     train_loss.mean(axis=0) + train_loss.std(axis=0),
                     alpha=0.25)
    plt.plot(epochs, val_loss.mean(axis=0), label="Validation Loss")
    plt.fill_between(epochs,
                     val_loss.mean(axis=0) - val_loss.std(axis=0),
                     val_loss.mean(axis=0) + val_loss.std(axis=0),
                     alpha=0.25)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss - MLP Fusion")
    plt.legend()
    
    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc.mean(axis=0), label="Train Accuracy")
    plt.fill_between(epochs,
                     train_acc.mean(axis=0) - train_acc.std(axis=0),
                     train_acc.mean(axis=0) + train_acc.std(axis=0),
                     alpha=0.25)
    plt.plot(epochs, val_acc.mean(axis=0), label="Validation Accuracy")
    plt.fill_between(epochs,
                     val_acc.mean(axis=0) - val_acc.std(axis=0),
                     val_acc.mean(axis=0) + val_acc.std(axis=0),
                     alpha=0.25)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy - MLP Fusion")
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=650, bbox_inches="tight")
    plt.show()

plot_average_training_curves(all_histories)

# Confusion Matrix and Classification Report
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

model = EarlyFusionWithMLPMeta(input_dim_meta=input_dim_meta, num_classes=num_classes, backbone='mobilevit_xxs').to(device)
model.load_state_dict(best_overall_model)
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, metadata, labels in test_loader:
        images, metadata = images.to(device), metadata.to(device)
        outputs = model(images, metadata)
        preds = outputs.argmax(dim=1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - MLP Fusion', fontweight='bold')
plt.ylabel('True Label', fontweight='bold')
plt.xlabel('Predicted Label', fontweight='bold')
plt.tight_layout()
plt.show()

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names, zero_division=0))

In [ ]:
import time
from ptflops import get_model_complexity_info

# =========================================================
# Parameter Count
# =========================================================
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

params_m = count_parameters(model) / 1e6
print(f"Total Trainable Parameters: {params_m:.3f} M")

# =========================================================
# FLOPs with Fusion Wrapper
# =========================================================
class FusionWrapper(nn.Module):
    def __init__(self, model, meta_dim):
        super().__init__()
        self.model = model
        self.meta_dim = meta_dim
    
    def forward(self, x):
        B = x.size(0)
        dummy_meta = torch.randn(B, self.meta_dim, device=x.device)
        return self.model(x, dummy_meta)

wrapped_model = FusionWrapper(model, input_dim_meta)

with torch.no_grad():
    flops, params = get_model_complexity_info(
        wrapped_model,
        (3, 224, 224),
        as_strings=False,
        print_per_layer_stat=False,
        verbose=False
    )

flops_g = flops / 1e9
print(f"FLOPs: {flops_g:.3f} GFLOPs")

# =========================================================
# GPU Latency
# =========================================================
def measure_gpu_latency(model, meta_dim, device, runs=200, warmup=30):
    if not torch.cuda.is_available():
        print("CUDA not available")
        return None, None
    
    model.eval()
    model.to(device)
    
    dummy_img = torch.randn(1, 3, 224, 224, device=device)
    dummy_meta = torch.randn(1, meta_dim, device=device)
    
    # Warmup
    for _ in range(warmup):
        _ = model(dummy_img, dummy_meta)
    
    torch.cuda.synchronize()
    
    # Measure
    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)
    times = []
    
    for _ in range(runs):
        start_event.record()
        _ = model(dummy_img, dummy_meta)
        end_event.record()
        torch.cuda.synchronize()
        times.append(start_event.elapsed_time(end_event))
    
    times = np.array(times)
    avg_latency_ms = times.mean()
    std_latency_ms = times.std()
    fps = 1000.0 / avg_latency_ms
    
    print(f"GPU Latency: {avg_latency_ms:.3f} ± {std_latency_ms:.3f} ms  |  FPS: {fps:.1f} (avg over {runs} runs)")
    return avg_latency_ms, std_latency_ms

# =========================================================
# CPU Latency
# =========================================================
def measure_cpu_latency(model, meta_dim, runs=200, warmup=30):
    model.eval()
    model.to('cpu')
    
    dummy_img = torch.randn(1, 3, 224, 224)
    dummy_meta = torch.randn(1, meta_dim)
    
    # Warmup
    for _ in range(warmup):
        _ = model(dummy_img, dummy_meta)
    
    # Measure
    times = []
    for _ in range(runs):
        start = time.time()
        _ = model(dummy_img, dummy_meta)
        end = time.time()
        times.append((end - start) * 1000)
    
    times = np.array(times)
    avg_latency_ms = times.mean()
    std_latency_ms = times.std()
    fps = 1000.0 / avg_latency_ms
    print(f"CPU Latency: {avg_latency_ms:.3f} ± {std_latency_ms:.3f} ms  |  FPS: {fps:.1f} (avg over {runs} runs)")
    return avg_latency_ms, std_latency_ms

# =========================================================
# Run Benchmarks
# =========================================================
print("\n" + "="*60)
print("Complexity Analysis - MLP Fusion")
print("="*60)

gpu_latency, gpu_std = measure_gpu_latency(model, input_dim_meta, device)
cpu_latency, cpu_std = measure_cpu_latency(model, input_dim_meta)

print("\n" + "="*60)
print("Summary")
print("="*60)
print(f"Parameters: {params_m:.3f} M")
print(f"FLOPs: {flops_g:.3f} GFLOPs")
if gpu_latency:
    gpu_fps = 1000.0 / gpu_latency
    print(f"GPU Latency: {gpu_latency:.3f} ± {gpu_std:.3f} ms  |  FPS: {gpu_fps:.1f}")
cpu_fps = 1000.0 / cpu_latency
print(f"CPU Latency: {cpu_latency:.3f} ± {cpu_std:.3f} ms  |  FPS: {cpu_fps:.1f}")
